## for all subjects, list xml files, compare to pipelines, clean duplicates, and output list of .xml rstring with their corresponding pipeline name

In [1]:
from helpers import *
%matplotlib inline
config.DATADIR                 = '/scratch/duboisjx/data/HCP/MRI'

In [22]:
subjects          = [s for s in sorted(os.listdir(config.DATADIR)) if unicode(s,'utf8').isnumeric()]
sessions          = ['REST1','REST2']
fmriFileTemplates = ['#fMRIrun#.nii.gz','#fMRIrun#_Atlas.dtseries.nii','#fMRIrun#_Atlas_MSMAll.dtseries.nii']
pipelines         = sorted(config.operationDict.keys())
outFile           = op.join(config.DATADIR,"listPrepro.txt")

In [24]:
try:
    remove(outFile)
except OSError:
    pass
for config.fmriFileTemplate in fmriFileTemplates:
    tmp = config.fmriFileTemplate.split('.')
    for config.subject in subjects:
        for session in sessions:
            fmriRuns                       = ['rfMRI_'+session+'_LR','rfMRI_'+session+'_RL']
            for config.fmriRun in fmriRuns:
                if tmp[1]=='nii':
                    config.isCifti = False
                elif tmp[1]=='dtseries':
                    config.isCifti = True
                else:
                    print 'unknown file extension'
                if config.isCifti:
                    config.ext = '.dtseries.nii'
                else:
                    config.ext = '.nii.gz'

                for config.pipelineName in pipelines:
                    if config.pipelineName in ['SiegelA','SiegelB']:
                        config.useFIX              = True
                    else:
                        config.useFIX              = False

                    config.Operations              = config.operationDict[config.pipelineName]
                    config.suffix = '_hp2000_clean' if config.useFIX else '' 

                    if hasattr(config,'fmriFileTemplate'):
                        #e.g. config.fmriFileTemplate =  #fMRIrun#_Atlas_MSMAll.dtseries.nii
                        config.fmriFile = op.join(buildpath(), config.fmriFileTemplate.replace('#fMRIrun#', config.fmriRun))
                    else:
                        if config.isCifti:
                            config.fmriFile = op.join(buildpath(), config.fmriRun+'_Atlas'+config.suffix+'.dtseries.nii')
                        else:
                            config.fmriFile = op.join(buildpath(), config.fmriRun+config.suffix+'.nii.gz')

                    # set up operations list
                    config.sortedOperations = sorted(config.Operations, key=operator.itemgetter(1))
                    config.steps = {}
                    config.Flavors = {}
                    cstep = 0
                    # If requested, scrubbing is performed first, before any denoising step
                    scrub_idx = -1
                    curr_idx = -1
                    for opr in config.sortedOperations:
                        curr_idx = curr_idx+1
                        if opr[0] == 'Scrubbing' and opr[1] != 1 and opr[1] != 0:
                            scrub_idx = opr[1]
                            break
                    if scrub_idx != -1:        
                        for opr in config.sortedOperations:  
                            if opr[1] != 0 and opr[1] <= scrub_idx:
                                opr[1] = opr[1]+1
                        config.sortedOperations[curr_idx][1] = 1
                        config.sortedOperations = sorted(config.Operations, key=operator.itemgetter(1))
                    for opr in config.sortedOperations:
                        if opr[1]==0:
                            continue
                        else:
                            if opr[1]!=cstep:
                                cstep=cstep+1
                                config.steps[cstep] = [opr[0]]
                                config.Flavors[cstep] = [opr[2]]
                            else:
                                config.steps[cstep].append(opr[0])
                                config.Flavors[cstep].append(opr[2])

                    precomputed = checkXML(config.fmriFile,config.steps,config.Flavors,buildpath()) 

                    if precomputed[0] is not None:
                        with open(outFile, "a") as myfile:
                            myfile.write('{},{},{},{},{}\n'.format(config.subject,config.fmriRun,config.fmriFileTemplate,config.pipelineName,get_rcode(precomputed[0])))
                        print '{},{},{},{},{}'.format(config.subject,config.fmriRun,config.fmriFileTemplate,config.pipelineName,get_rcode(precomputed[0])) 
                        for p in precomputed[1:]:
                            try:
                                remove(p)
                            except OSError:
                                pass
                            try:
                                remove(op.join(buildpath(),get_rcode(p)+'.xml'))
                            except OSError:
                                pass
                            for parcellation in ['shen2013','HO110_25_GM','HO110_25','HarvardOxford25','Glasser_Aseg_SUIT']:
                                try:
                                    for fdel in glob.glob(op.join(buildpath(),parcellation,config.fmriRun+config.ext,'*'+get_rcode(p)+'*')):
                                        remove(fdel)
                                except OSError:
                                    pass


100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric1,ou9eX9e7
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric13,rRK22gPb
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric14,p8LHIavK
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric2,SDD5vbFH
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric3,acbMamfT
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric4,7c91WNnd
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric5,HSUD1ua3
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric6,LjlubGdD
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Ciric9,0QFQFgbs
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Finn,biLe7IpX
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Finn_noTsmooth,aXpJRDo6
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Gordon1,oAHTf2Kb
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,Gordon2,0YA6i3ep
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,SiegelA,tf6jkXTx
100206,rfMRI_REST1_LR,#fMRIrun#.nii.gz,SiegelB,U8Jik09B
100206,rfMRI_REST1_RL,#fMRIrun#.nii.gz,Ciric1,7lX7GgTH
100206,rfMRI_REST1_RL,#fMRIrun#.nii.gz,Ciric13,wVWOTgNp
100206,rfMRI_REST1_RL,#fMRIrun#.nii.gz,Ciric14,vknLM